In [43]:
from google.cloud.storage.blob import Blob
from google.cloud import storage
import os
import sys
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import csv

In [44]:
df = pd.read_csv('https://storage.googleapis.com/geolocation_data_project_cm/distances_CompA.csv', delimiter=',', encoding='utf-8', header=0)
print(df.head(20))
print(df.info())

                                    ORGANISATION_NAME  REGISTRATION_NO  \
0                    THE SUN INSURANCE OFFICE LIMITED               11   
1                     COMPTOIR D' ESCOMPTE DE NICOSIE               13   
2                                    SIMMONS BROTHERS               32   
3                                  NEW STAR SHOE SHOP              118   
4                           CHARALAMBOS PIERI AND SON               17   
5   ΚΥΠΡΙΑΚΟΝ ΕΜΠΟΡΙΚΟΝ ΚΑΙ ΒΙΟΜΗΧΑΝΙΚΟΝ ΕΠΙΜΕΛΗΤΗ...               19   
6   ΕΜΠΟΡΙΚΟΝ ΚΑΙ ΒΙΟΜΗΧΑΝΙΚΟΝ ΕΠΙΜΕΛΗΤΗΡΙΟΝ ΛΕΥΚΩ...              502   
7   Ο ΚΥΠΡΙΑΚΟΣ ΣΥΝΔΕΣΜΟΣ ΙΔΙΟΚΤΗΤΩΝ ΑΚΙΝΗΤΩΝ ΛΙΜΙΤΕΔ             1293   
8                CYPRUS-AMERICAN BUSINESS ASSOCIATION            17284   
9                                               CYABA            17285   
10                                      AMCHAM CYPRUS            35841   
11                                   C.J. CACOYIANNIS               28   
12                                    

In [45]:
#Following investigation, Cyprus lies at a latitude of 34˚33˚ - 35˚34˚ North and longitude 32˚16˚ - 34˚37˚ East. 
#Therefore we will exclude addresses that do not fall in the valid range of Cy geocoordinates
#lat1, lon1, lat2, lon2 = 35.2, 34.37, 34.33, 32.16
#Further investigation and given that we were losing 2 service points of the existing service points of CompA,
# as Cyprus cannot be described by a perfect rectangular shape, 
#the below latitude and longitude extremes will be used 
lat1, lon1, lat2, lon2 = 35.2, 34.85, 34.33, 32.16
dist_df = df[(df['lat'] <= lat1) & (df['lat'] >= lat2) & (df['lng'] <= lon1) & (df['lng'] >= lon2)]
dist_df

ORGANISATION_NAME  REGISTRATION_NO  \
0       THE SUN INSURANCE OFFICE LIMITED               11   
1        COMPTOIR D' ESCOMPTE DE NICOSIE               13   
2                       SIMMONS BROTHERS               32   
3                     NEW STAR SHOE SHOP              118   
4              CHARALAMBOS PIERI AND SON               17   
...                                  ...              ...   
218733   BEAUTY STUDIO BY THEKLA LIMITED           441880   
218734                      ZI. RA 1 LTD           441941   
218735                 ACLA AUDITORS LTD           441954   
218736         ΣΥΛΒΕΣΤΡΟΥ ΚΑΙ ΠΗΛΑΔΗ ΛΤΔ           442013   
218737                       ILLIDAN LTD           442033   

       ORGANISATION_TYPE_CODE  ORGANISATION_TYPE ORGANISATION_STATUS  \
0                           O  Αλλοδαπή Εταιρεία        Εγγεγραμμένη   
1                           B  Εμπορική Επωνυμία        Εγγεγραμμένη   
2                           B  Εμπορική Επωνυμία        Εγγεγραμμένη   
3                           B  Εμπορική Επωνυμία        Εγγεγραμμένη   
4                           B  Εμπορική Επωνυμία        Εγγεγραμμένη   
...                       ...                ...                 ...   
218733                      C           Εταιρεία        Εγγεγραμμένη   
218734                      C           Εταιρεία        Εγγεγραμμένη   
218735                      C           Εταιρεία        Εγγεγραμμένη   
218736                      C           Εταιρεία        Εγγεγραμμένη   
218737                      C           Εταιρεία        Εγγεγραμμένη   

        address_seq_no                     ADDRESS_L1  \
0               281510     ΠΡΙΓΚΙΠΙΣΣΗΣ ΝΤΕ ΤΥΡΑΣ 12    
1               274852                            NaN   
2               274852                            NaN   
3               274852                            NaN   
4               262535                ARISTIDES ROAD    
...                ...                            ...   
218733          267985  ΠΑΝΑΓΙΩΤΗ ΑΝΑΓΝΩΣΤΟΠΟΥΛΟΥ 68    
218734          565102                  ΓΛΑΔΣΤΩΝΟΣ 1    
218735          439357              ΠΕΝΤΑΔΑΚΤΥΛΟΥ 21    
218736          352550                   ΟΔΥΣΣΕΩΣ 10    
218737          195348               ΑΡΓΥΡΗ ΓΑΒΑΛΑ 3    

                            ADDRESS_L3  \
0                            ΛΕΥΚΩΣΙΑ    
1                            ΛΕΥΚΩΣΙΑ    
2                            ΛΕΥΚΩΣΙΑ    
3                            ΛΕΥΚΩΣΙΑ    
4                            ΛΕΥΚΩΣΙΑ    
...                                ...   
218733    ΚΑΤΩ ΠΟΛΕΜΙΔΙΑ 4152 ΛΕΜΕΣΟΣ    
218734                   6023 ΛΑΡΝΑΚΑ    
218735  ΑΓΙΟΙ ΤΡΙΜΙΘΙΑΣ 2671 ΛΕΥΚΩΣΙΑ    
218736        ΤΕΡΣΕΦΑΝΟΥ 7562 ΛΑΡΝΑΚΑ    
218737                   3112 ΛΕΜΕΣΟΣ    

                                             FULL ADDRESS  \
0                    ΠΡΙΓΚΙΠΙΣΣΗΣ ΝΤΕ ΤΥΡΑΣ 12  ΛΕΥΚΩΣΙΑ    
1                                               ΛΕΥΚΩΣΙΑ    
2                                               ΛΕΥΚΩΣΙΑ    
3                                               ΛΕΥΚΩΣΙΑ    
4                               ARISTIDES ROAD  ΛΕΥΚΩΣΙΑ    
...                                                   ...   
218733  ΠΑΝΑΓΙΩΤΗ ΑΝΑΓΝΩΣΤΟΠΟΥΛΟΥ 68  ΚΑΤΩ ΠΟΛΕΜΙΔΙΑ 4...   
218734                        ΓΛΑΔΣΤΩΝΟΣ 1  6023 ΛΑΡΝΑΚΑ    
218735   ΠΕΝΤΑΔΑΚΤΥΛΟΥ 21  ΑΓΙΟΙ ΤΡΙΜΙΘΙΑΣ 2671 ΛΕΥΚΩΣΙΑ    
218736              ΟΔΥΣΣΕΩΣ 10  ΤΕΡΣΕΦΑΝΟΥ 7562 ΛΑΡΝΑΚΑ    
218737                     ΑΡΓΥΡΗ ΓΑΒΑΛΑ 3  3112 ΛΕΜΕΣΟΣ    

                                lat_lng  ...     51      52      53      54  \
0              (35.1602214, 33.3604734)  ...  64.01   94.93   94.19   95.48   
1       (35.1855659, 33.38227639999999)  ...  67.45   97.95   97.22   98.51   
2       (35.1855659, 33.38227639999999)  ...  67.45   97.95   97.22   98.51   
3       (35.1855659, 33.38227639999999)  ...  67.45   97.95   97.22   98.51   
4       (35.1855659, 33.38227639999999)  ...  67.45   97.95   97.22   98.51   
...                         

In [46]:
msr_df = dist_df.select_dtypes(include=['number'])
msr_df = msr_df.drop(['lat_txt','lng_txt','lat','lng','address_seq_no','REGISTRATION_NO'], axis=1)
msr_df['min_dist'] = msr_df.min(axis='columns')
msr_df['min_sp'] = msr_df.idxmin(axis='columns')
msr_df

Valid Seq      1      2      3      4      5      6      7      8  \
0          281510   6.45   1.19   0.67   2.54   3.87   1.92  16.45   3.19   
1          274852   9.05   4.50   2.80   5.98   6.28   4.52  19.01   4.16   
2          274852   9.05   4.50   2.80   5.98   6.28   4.52  19.01   4.16   
3          274852   9.05   4.50   2.80   5.98   6.28   4.52  19.01   4.16   
4          262535   9.05   4.50   2.80   5.98   6.28   4.52  19.01   4.16   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
218733     267985  58.00  61.16  62.86  59.69  60.67  61.62  51.36  62.79   
218734     565102  30.16  34.97  36.14  35.60  38.79  37.40  23.20  32.98   
218735     439357  15.29  14.03  14.91  12.27  11.17  12.92  20.02  16.93   
218736     352550  31.67  36.94  38.43  37.11  40.24  39.24  22.58  35.61   
218737     195348  56.46  59.87  61.61  58.47  59.63  60.48  49.36  61.36   

            9  ...      53      54      55      56      57      58     59  \
0        2.36  ...   94.19   95.48   95.59   95.13   94.30   85.98   1.51   
1        5.08  ...   97.22   98.51   98.46   98.17   97.37   88.41   3.67   
2        5.08  ...   97.22   98.51   98.46   98.17   97.37   88.41   3.67   
3        5.08  ...   97.22   98.51   98.46   98.17   97.37   88.41   3.67   
4        5.08  ...   97.22   98.51   98.46   98.17   97.37   88.41   3.67   
...       ...  ...     ...     ...     ...     ...     ...     ...    ...   
218733  61.13  ...   52.68   53.75   59.70   53.13   51.65   65.13  62.23   
218734  37.53  ...  109.12  110.38  113.35  109.87  108.65  109.40  34.37   
218735  12.36  ...   79.81   81.10   81.12   80.76   79.96   71.85  15.44   
218736  39.23  ...  101.75  102.98  106.49  102.45  101.17  103.95  36.69   
218737  60.01  ...   56.50   57.57   63.49   56.95   55.47   68.59  60.89   

           60  min_dist  min_sp  
0       58.34      0.67       3  
1       61.76      0.57      17  
2       61.76      0.57      17  
3       61.76      0.57      17  
4       61.76      0.57      17  
...       ...       ...     ...  
218733   6.04      1.01      50  
218734  56.42      0.50      30  
218735  48.58      3.93      21  
218736  47.45      3.26      31  
218737   2.83      0.56      39  

[217788 rows x 63 columns]

In [47]:
#Keep only the the interesting columns
serv_pts_df = pd.DataFrame(msr_df, columns=['min_dist', 'min_sp'])
serv_pts_df['min_sp'] = serv_pts_df['min_sp'].astype('int64')
serv_pts_df

min_dist  min_sp
0           0.67       3
1           0.57      17
2           0.57      17
3           0.57      17
4           0.57      17
...          ...     ...
218733      1.01      50
218734      0.50      30
218735      3.93      21
218736      3.26      31
218737      0.56      39

[217788 rows x 2 columns]

In [48]:
serv_pts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217788 entries, 0 to 218737
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   min_dist  217788 non-null  float64
 1   min_sp    217788 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 5.0 MB


In [49]:
#Concatenate dataframes to get the closest service point per customer
cust_df = pd.DataFrame(dist_df, columns=['ORGANISATION_NAME','address_seq_no', 'FULL ADDRESS', 'lat', 'lng'])
cust_sp_df = pd.concat([cust_df,serv_pts_df],axis=1)
cust_sp_df.rename(columns={'address_seq_no':'AddressID'}, inplace=True)
cust_sp_df.rename(columns={'min_dist':'MinDistTo_CompA_SP'}, inplace=True)
cust_sp_df.rename(columns={'min_sp':'CompA_SP'}, inplace=True)
cust_sp_df

ORGANISATION_NAME  AddressID  \
0       THE SUN INSURANCE OFFICE LIMITED     281510   
1        COMPTOIR D' ESCOMPTE DE NICOSIE     274852   
2                       SIMMONS BROTHERS     274852   
3                     NEW STAR SHOE SHOP     274852   
4              CHARALAMBOS PIERI AND SON     262535   
...                                  ...        ...   
218733   BEAUTY STUDIO BY THEKLA LIMITED     267985   
218734                      ZI. RA 1 LTD     565102   
218735                 ACLA AUDITORS LTD     439357   
218736         ΣΥΛΒΕΣΤΡΟΥ ΚΑΙ ΠΗΛΑΔΗ ΛΤΔ     352550   
218737                       ILLIDAN LTD     195348   

                                             FULL ADDRESS        lat  \
0                    ΠΡΙΓΚΙΠΙΣΣΗΣ ΝΤΕ ΤΥΡΑΣ 12  ΛΕΥΚΩΣΙΑ   35.160221   
1                                               ΛΕΥΚΩΣΙΑ   35.185566   
2                                               ΛΕΥΚΩΣΙΑ   35.185566   
3                                               ΛΕΥΚΩΣΙΑ   35.185566   
4                               ARISTIDES ROAD  ΛΕΥΚΩΣΙΑ   35.185566   
...                                                   ...        ...   
218733  ΠΑΝΑΓΙΩΤΗ ΑΝΑΓΝΩΣΤΟΠΟΥΛΟΥ 68  ΚΑΤΩ ΠΟΛΕΜΙΔΙΑ 4...  34.685159   
218734                        ΓΛΑΔΣΤΩΝΟΣ 1  6023 ΛΑΡΝΑΚΑ   34.918222   
218735   ΠΕΝΤΑΔΑΚΤΥΛΟΥ 21  ΑΓΙΟΙ ΤΡΙΜΙΘΙΑΣ 2671 ΛΕΥΚΩΣΙΑ   35.114799   
218736              ΟΔΥΣΣΕΩΣ 10  ΤΕΡΣΕΦΑΝΟΥ 7562 ΛΑΡΝΑΚΑ   34.854814   
218737                     ΑΡΓΥΡΗ ΓΑΒΑΛΑ 3  3112 ΛΕΜΕΣΟΣ   34.678632   

              lng  MinDistTo_CompA_SP  CompA_SP  
0       33.360473                0.67         3  
1       33.382276                0.57        17  
2       33.382276                0.57        17  
3       33.382276                0.57        17  
4       33.382276                0.57        17  
...           ...                 ...       ...  
218733  33.000279                1.01        50  
218734  33.620063                0.50        30  
218735  33.211395                3.93        21  
218736  33.546889                3.26        31  
218737  33.041305                0.56        39  

[217788 rows x 7 columns]

In [52]:
cust_sp_df.to_csv('CompA_MinSP_AllData.csv', encoding='utf-8')

In [56]:
CompA_Loc = pd.read_csv('https://storage.googleapis.com/geolocation_data_project_cm/CompA_Locs.csv', delimiter=';', encoding='utf-8')
CompA_Loc.dropna(inplace=True)
CompA_Loc['AA'] = CompA_Loc['AA'].astype('int64')
CompA_Loc.rename(columns={'AA': 'Service Point'}, inplace=True)
CompA_Loc

Service Point                                       Full Address  \
0               1              Ελευθερίου Βενιζέλου 1, Λευκωσία 2235   
1               2                Λεωφ. Ακροπόλεως 58Α, Λευκωσία 2012   
2               3                   Λεωφ. Μακαρίου 39, Λευκωσία 1065   
3               4                 Λεωφ. Στροβόλου 116, Λευκωσία 2042   
4               5                   28ης Οκτωβρίου 8A, Λευκωσία 2414   
5               6         Γωνία Αγ.Προκοπίου&Ποσειδ 1, Λευκωσία 2406   
6               7            Λεωφ. Λεμεσού 38, Πέρα Χωρίο Νήσου 2572   
7               8          Λεωφ. Κυρηνείας 97, Πλατύ Αγλαντζιάς 2113   
8               9                       Βυζαντίου 5A, Στρόβολος 2064   
9              10                       Μακράσυκας 1, Στρόβολος 2034   
10             11                  Θεοδόση Πιερίδη 14-16, Τσέρι 2480   
11             12  Λεωφ. Αρχ. Μακαρίου 81, Αγ. Ελευθέριος, Λατσιά...   
12             13      Διγενή Ακρίτα 11, Αγ. Αντώνιος, Λευκωσία 1055   
13             14           Αγ.Γεωργίου & Δ.Χάματσου, Ανθούπολη 2312   
14             15  Γωνία Λεωφ. Αθαλάσσας 154E, Δασούπολη, Λευκωσί...   
15             16                     Μαρκου Δράκου 21, Ευρύχου 2831   
16             17                   Λεωφ. Καντάρας 14, Λευκωσία 1041   
17             18                Αγίου Στυλιανιού 1, Λακατάμεια 2324   
18             19                  Λεωφ. Κένεντυ 67,  Στρόβολος 1076   
19             20                   Φανερωμένης 86-90, Λευκωσία 1011   
20             21       Λεωφ. Γρ. Αυξεντίου 3A, Κοκκινοτριμιθιά 2660   
21             22                 Λεωφ. Κυρ. Μάτση 50, Λευκωσία 2368   
22             23                       Λεωφ. Νησί 58,Αγία Νάπα 5330   
23             24                   1ης Απριλίου 160, Παραλίμνι 5280   
24             25                      Ελευθερίας 1, Ξυλοτύμπου 7510   
25             26                 Αναστάση Μανώλη 44, Ξυλοφάγου 7520   
26             27                    Λεωφ. Μακαρίου 1, Αθηαίνου 7600   
27             28              Λεωφ. Μακαρίου III 30, Αραδίππου 7104   
28             29            Λεωφ. Αρχ.Μακαρίου ΙΙΙ 45, Λάρνακα 6017   
29             30               Λεωφ. Γρ. Αυξεντίου 42, Λάρνακα 6021   
30             31                        Αρχ. Μακαρίου 27, Κίτι 7550   
31             32           Κωνστ.Παλαιολόγου & Τήνου , Λάρνακα 6036   
32             33                  Ζήνωνος Κιτιέως 40, Λειβάδια 7060   
33             34         Λεωφ. Σπύρου Κυπριανού 41-43, Λάρνακα 6051   
34             35                               Πάφου 13, Ερήμη 4620   
35             36                  Λεωφ.Μακαρίου Γ 258, Λεμεσός 3105   
36             37         Παν.Ευαγγελίστριας 28, Κάτω Πολεμίδια 4156   
37             38                   Αγίας Φυλάξεως 230, Κάψαλος 3083   
38             39                          Σαριπόλου 1, Λεμεσός 3036   
39             40         Λεωφ. Φραγκλίνου Ρούσβελτ 86, Λεμεσός 3011   
40             41          Λεωφ. Αρχ. Μακαρίου 121-123, Λεμεσός 3021   
41             42                  Μακεδονίας 54, Μέσα Γειτονιά 4001   
42             43                     Νίκου Παττίχη 39, Λεμεσός 3071   
43             44                        Οδού Πάφου 51, Λεμεσός 3052   
44             45      Αγ.Ζώνης & Θεσσαλονίκης, Πεντάδρομος, Λεμεσός   
45             46                            Ολύμπου 6, Πλάτρες 4820   
46             47                     Γλαύκου Κληρίδη 43, Αγρός 4860   
47             48                        Ακαδημίας 86B, Λεμεσός 4043   
48             49                   Γεωργίου Α' 81, Γερμασόγεια 4047   
49             50  Θεόδωρου Ποταμιάνου 48, Συνοικισμός Μακαρίου Ι...   
50             51                    Ηλία Καννάουρου 33, Ύψωνας 4180   
51             52                    οδός Ακαμαντίδος 26, Πάφος 8016   
52             53                Ελευθερίου Βενιζέλου 49, Πάφος 8021   
53             54             E.Παλληκαρίδη&Α.Τσιέλεπου , Πάφος 8010   
54             55  Λεωφόρος Κόλπου Κοραλίων 

In [57]:
#Some baseline counts regarding existing physical branch network of Company A
customers_per_sp = serv_pts_df.groupby("min_sp").count().reset_index()
customers_per_sp.columns = ['Service Point', 'NoOfCustomers']
customers_per_sp['Service Point'] = customers_per_sp['Service Point'].astype('int64')
customers_per_sp['NoOfCustomers'] = customers_per_sp['NoOfCustomers'].astype('int64')
print(customers_per_sp)
join_df = pd.merge(CompA_Loc, customers_per_sp, on="Service Point",how="left")
join_df['NoOfCustomers'].fillna(0, inplace=True)
join_df['NoOfCustomers'] = join_df['NoOfCustomers'].astype('int64')
join_df.sort_values('Service Point', ascending=True)

    Service Point  NoOfCustomers
0               1           1707
1               2           4052
2               3          23857
3               4           5558
4               5           4740
5               6           6055
6               7           3058
7               8           2629
8               9           1669
9              10           1798
10             11            681
11             12           2585
12             13          11184
13             14           1649
14             15           2222
15             16            327
16             17           5151
17             18           3773
18             19           4379
19             20           6783
20             21           1475
21             22           3163
22             23           2121
23             24           6225
24             25           1668
25             26           1517
26             27            655
27             28           1019
28             29           3157
29        

Service Point                                       Full Address  \
0               1              Ελευθερίου Βενιζέλου 1, Λευκωσία 2235   
1               2                Λεωφ. Ακροπόλεως 58Α, Λευκωσία 2012   
2               3                   Λεωφ. Μακαρίου 39, Λευκωσία 1065   
3               4                 Λεωφ. Στροβόλου 116, Λευκωσία 2042   
4               5                   28ης Οκτωβρίου 8A, Λευκωσία 2414   
5               6         Γωνία Αγ.Προκοπίου&Ποσειδ 1, Λευκωσία 2406   
6               7            Λεωφ. Λεμεσού 38, Πέρα Χωρίο Νήσου 2572   
7               8          Λεωφ. Κυρηνείας 97, Πλατύ Αγλαντζιάς 2113   
8               9                       Βυζαντίου 5A, Στρόβολος 2064   
9              10                       Μακράσυκας 1, Στρόβολος 2034   
10             11                  Θεοδόση Πιερίδη 14-16, Τσέρι 2480   
11             12  Λεωφ. Αρχ. Μακαρίου 81, Αγ. Ελευθέριος, Λατσιά...   
12             13      Διγενή Ακρίτα 11, Αγ. Αντώνιος, Λευκωσία 1055   
13             14           Αγ.Γεωργίου & Δ.Χάματσου, Ανθούπολη 2312   
14             15  Γωνία Λεωφ. Αθαλάσσας 154E, Δασούπολη, Λευκωσί...   
15             16                     Μαρκου Δράκου 21, Ευρύχου 2831   
16             17                   Λεωφ. Καντάρας 14, Λευκωσία 1041   
17             18                Αγίου Στυλιανιού 1, Λακατάμεια 2324   
18             19                  Λεωφ. Κένεντυ 67,  Στρόβολος 1076   
19             20                   Φανερωμένης 86-90, Λευκωσία 1011   
20             21       Λεωφ. Γρ. Αυξεντίου 3A, Κοκκινοτριμιθιά 2660   
21             22                 Λεωφ. Κυρ. Μάτση 50, Λευκωσία 2368   
22             23                       Λεωφ. Νησί 58,Αγία Νάπα 5330   
23             24                   1ης Απριλίου 160, Παραλίμνι 5280   
24             25                      Ελευθερίας 1, Ξυλοτύμπου 7510   
25             26                 Αναστάση Μανώλη 44, Ξυλοφάγου 7520   
26             27                    Λεωφ. Μακαρίου 1, Αθηαίνου 7600   
27             28              Λεωφ. Μακαρίου III 30, Αραδίππου 7104   
28             29            Λεωφ. Αρχ.Μακαρίου ΙΙΙ 45, Λάρνακα 6017   
29             30               Λεωφ. Γρ. Αυξεντίου 42, Λάρνακα 6021   
30             31                        Αρχ. Μακαρίου 27, Κίτι 7550   
31             32           Κωνστ.Παλαιολόγου & Τήνου , Λάρνακα 6036   
32             33                  Ζήνωνος Κιτιέως 40, Λειβάδια 7060   
33             34         Λεωφ. Σπύρου Κυπριανού 41-43, Λάρνακα 6051   
34             35                               Πάφου 13, Ερήμη 4620   
35             36                  Λεωφ.Μακαρίου Γ 258, Λεμεσός 3105   
36             37         Παν.Ευαγγελίστριας 28, Κάτω Πολεμίδια 4156   
37             38                   Αγίας Φυλάξεως 230, Κάψαλος 3083   
38             39                          Σαριπόλου 1, Λεμεσός 3036   
39             40         Λεωφ. Φραγκλίνου Ρούσβελτ 86, Λεμεσός 3011   
40             41          Λεωφ. Αρχ. Μακαρίου 121-123, Λεμεσός 3021   
41             42                  Μακεδονίας 54, Μέσα Γειτονιά 4001   
42             43                     Νίκου Παττίχη 39, Λεμεσός 3071   
43             44                        Οδού Πάφου 51, Λεμεσός 3052   
44             45      Αγ.Ζώνης & Θεσσαλονίκης, Πεντάδρομος, Λεμεσός   
45             46                            Ολύμπου 6, Πλάτρες 4820   
46             47                     Γλαύκου Κληρίδη 43, Αγρός 4860   
47             48                        Ακαδημίας 86B, Λεμεσός 4043   
48             49                   Γεωργίου Α' 81, Γερμασόγεια 4047   
49             50  Θεόδωρου Ποταμιάνου 48, Συνοικισμός Μακαρίου Ι...   
50             51                    Ηλία Καννάουρου 33, Ύψωνας 4180   
51             52                    οδός Ακαμαντίδος 26, Πάφος 8016   
52             53                Ελευθερίου Βενιζέλου 49, Πάφος 8021   
53             54             E.Παλληκαρίδη&Α.Τσιέλεπου , Πάφος 8010   
54             55  Λεωφόρος Κόλπου Κοραλίων 

In [58]:
#Statistics per service point
stats_per_sp = serv_pts_df.groupby(['min_sp']).aggregate([np.min,np.max,np.mean,np.median]).reset_index()
stats_per_sp.columns = stats_per_sp.columns.map('_'.join)
stats_per_sp.rename(columns={'min_sp_':'Service Point'}, inplace=True)
stats_per_sp.rename(columns={'min_dist_amin':'Min Distance'}, inplace=True)
stats_per_sp.rename(columns={'min_dist_amax':'Max Distance'}, inplace=True)
stats_per_sp.rename(columns={'min_dist_mean':'Mean Distance'}, inplace=True)
stats_per_sp.rename(columns={'min_dist_median':'Median Distance'}, inplace=True)

stats_per_sp.sort_values('Service Point')

Service Point  Min Distance  Max Distance  Mean Distance  Median Distance
0               1          0.02          4.78       1.468336             0.66
1               2          0.02          1.23       0.534882             0.52
2               3          0.04          1.18       0.493258             0.48
3               4          0.03          2.34       0.655558             0.64
4               5          0.03          3.94       1.102515             1.03
5               6          0.16          1.21       0.706185             0.71
6               7          0.03         22.43       4.910602             3.04
7               8          0.06          9.25       1.006006             0.96
8               9          0.23          1.34       0.642720             0.69
9              10          0.07          2.06       0.853276             0.84
10             11          0.05         14.55       4.932482             6.10
11             12          0.04          3.97       1.431926             1.27
12             13          0.01          1.57       0.651675             0.74
13             14          0.02          7.78       1.548702             1.14
14             15          0.01          1.10       0.486170             0.50
15             16          0.35         26.66       8.797431             5.55
16             17          0.01          1.87       0.568618             0.57
17             18          0.09          2.93       1.235521             1.28
18             19          0.01          1.00       0.484371             0.47
19             20          0.02          2.69       0.462565             0.51
20             21          0.01         16.30       6.395153             5.47
21             22          0.04          3.23       0.760828             0.75
22             23          0.21          9.75       2.102192             1.73
23             24          0.02         20.74       2.680263             0.71
24             25          0.00          9.96       1.202830             1.15
25             26          0.01         22.11       5.817752             6.23
26             27          0.11         18.20       0.876183             0.18
27             28          0.14          2.09       0.569558             0.34
28             29          0.05          1.18       0.645480             0.67
29             30          0.02          3.26       0.841446             0.68
30             31          0.11         23.65       6.878007             8.58
31             32          0.03          3.56       0.640072             0.57
32             33          0.06          9.31       5.560522             4.61
33             34          0.03          1.82       0.598159             0.43
34             35          0.17         21.67       5.490067             2.08
35             36          0.02          0.93       0.464152             0.55
36             37          0.01         11.69       1.226610             0.93
37             38          0.05         11.67       5.281492             7.20
38             39          0.01          1.46       0.513446             0.56
39             40          0.09          8.77       0.760383             0.85
40             41          0.02          1.42       0.629923             0.63
41             42          0.02          2.13       0.498715             0.48
42             43          0.12          1.25       0.481033             0.43
43             44          0.00          1.99       0.789896             0.68
44             45          0.04          0.96       0.587912             0.67
45             46          0.09         21.56       4.210525             0.40
46             47          0.06         19.28       7.201584             6.62
47             48          0.04         16.14       1.122789             0.58
48             49          0.03         22.95       4.292675             1.22
49             50          0.05          1.10       0.349209             0.18
50             51         

In [59]:
stats_per_sp['95th Percentile'] = cust_sp_df.groupby('CompA_SP')['MinDistTo_CompA_SP'].quantile(0.95)
stats_per_sp['5th Percentile'] = cust_sp_df.groupby('CompA_SP')['MinDistTo_CompA_SP'].quantile(0.05)
stats_per_sp

Service Point  Min Distance  Max Distance  Mean Distance  Median Distance  \
0               1          0.02          4.78       1.468336             0.66   
1               2          0.02          1.23       0.534882             0.52   
2               3          0.04          1.18       0.493258             0.48   
3               4          0.03          2.34       0.655558             0.64   
4               5          0.03          3.94       1.102515             1.03   
5               6          0.16          1.21       0.706185             0.71   
6               7          0.03         22.43       4.910602             3.04   
7               8          0.06          9.25       1.006006             0.96   
8               9          0.23          1.34       0.642720             0.69   
9              10          0.07          2.06       0.853276             0.84   
10             11          0.05         14.55       4.932482             6.10   
11             12          0.04          3.97       1.431926             1.27   
12             13          0.01          1.57       0.651675             0.74   
13             14          0.02          7.78       1.548702             1.14   
14             15          0.01          1.10       0.486170             0.50   
15             16          0.35         26.66       8.797431             5.55   
16             17          0.01          1.87       0.568618             0.57   
17             18          0.09          2.93       1.235521             1.28   
18             19          0.01          1.00       0.484371             0.47   
19             20          0.02          2.69       0.462565             0.51   
20             21          0.01         16.30       6.395153             5.47   
21             22          0.04          3.23       0.760828             0.75   
22             23          0.21          9.75       2.102192             1.73   
23             24          0.02         20.74       2.680263             0.71   
24             25          0.00          9.96       1.202830             1.15   
25             26          0.01         22.11       5.817752             6.23   
26             27          0.11         18.20       0.876183             0.18   
27             28          0.14          2.09       0.569558             0.34   
28             29          0.05          1.18       0.645480             0.67   
29             30          0.02          3.26       0.841446             0.68   
30             31          0.11         23.65       6.878007             8.58   
31             32          0.03          3.56       0.640072             0.57   
32             33          0.06          9.31       5.560522             4.61   
33             34          0.03          1.82       0.598159             0.43   
34             35          0.17         21.67       5.490067             2.08   
35             36          0.02          0.93       0.464152             0.55   
36             37          0.01         11.69       1.226610             0.93   
37             38          0.05         11.67       5.281492             7.20   
38             39          0.01          1.46       0.513446             0.56   
39             40          0.09          8.77       0.760383             0.85   
40             41          0.02          1.42       0.629923             0.63   
41             42          0.02          2.13       0.498715             0.48   
42             43          0.12          1.25       0.481033             0.43   
43             44          0.00          1.99       0.789896             0.68   
44             45          0.04          0.96       0.587912             0.67   
45             46          0.09         21.56       4.210525             0.40   
46             47          0.06         19.28       7.201584             6.62   
47             48          0.04         16.14       1.122789             0.58   
48             49          0.03    

In [60]:
#Join statistics with the main service point information
join_df2 = pd.merge(join_df,stats_per_sp,on="Service Point",how="left")
join_df2.fillna(0, inplace=True)
join_df2.sort_values('Service Point', ascending=True)

Service Point                                       Full Address  \
0               1              Ελευθερίου Βενιζέλου 1, Λευκωσία 2235   
1               2                Λεωφ. Ακροπόλεως 58Α, Λευκωσία 2012   
2               3                   Λεωφ. Μακαρίου 39, Λευκωσία 1065   
3               4                 Λεωφ. Στροβόλου 116, Λευκωσία 2042   
4               5                   28ης Οκτωβρίου 8A, Λευκωσία 2414   
5               6         Γωνία Αγ.Προκοπίου&Ποσειδ 1, Λευκωσία 2406   
6               7            Λεωφ. Λεμεσού 38, Πέρα Χωρίο Νήσου 2572   
7               8          Λεωφ. Κυρηνείας 97, Πλατύ Αγλαντζιάς 2113   
8               9                       Βυζαντίου 5A, Στρόβολος 2064   
9              10                       Μακράσυκας 1, Στρόβολος 2034   
10             11                  Θεοδόση Πιερίδη 14-16, Τσέρι 2480   
11             12  Λεωφ. Αρχ. Μακαρίου 81, Αγ. Ελευθέριος, Λατσιά...   
12             13      Διγενή Ακρίτα 11, Αγ. Αντώνιος, Λευκωσία 1055   
13             14           Αγ.Γεωργίου & Δ.Χάματσου, Ανθούπολη 2312   
14             15  Γωνία Λεωφ. Αθαλάσσας 154E, Δασούπολη, Λευκωσί...   
15             16                     Μαρκου Δράκου 21, Ευρύχου 2831   
16             17                   Λεωφ. Καντάρας 14, Λευκωσία 1041   
17             18                Αγίου Στυλιανιού 1, Λακατάμεια 2324   
18             19                  Λεωφ. Κένεντυ 67,  Στρόβολος 1076   
19             20                   Φανερωμένης 86-90, Λευκωσία 1011   
20             21       Λεωφ. Γρ. Αυξεντίου 3A, Κοκκινοτριμιθιά 2660   
21             22                 Λεωφ. Κυρ. Μάτση 50, Λευκωσία 2368   
22             23                       Λεωφ. Νησί 58,Αγία Νάπα 5330   
23             24                   1ης Απριλίου 160, Παραλίμνι 5280   
24             25                      Ελευθερίας 1, Ξυλοτύμπου 7510   
25             26                 Αναστάση Μανώλη 44, Ξυλοφάγου 7520   
26             27                    Λεωφ. Μακαρίου 1, Αθηαίνου 7600   
27             28              Λεωφ. Μακαρίου III 30, Αραδίππου 7104   
28             29            Λεωφ. Αρχ.Μακαρίου ΙΙΙ 45, Λάρνακα 6017   
29             30               Λεωφ. Γρ. Αυξεντίου 42, Λάρνακα 6021   
30             31                        Αρχ. Μακαρίου 27, Κίτι 7550   
31             32           Κωνστ.Παλαιολόγου & Τήνου , Λάρνακα 6036   
32             33                  Ζήνωνος Κιτιέως 40, Λειβάδια 7060   
33             34         Λεωφ. Σπύρου Κυπριανού 41-43, Λάρνακα 6051   
34             35                               Πάφου 13, Ερήμη 4620   
35             36                  Λεωφ.Μακαρίου Γ 258, Λεμεσός 3105   
36             37         Παν.Ευαγγελίστριας 28, Κάτω Πολεμίδια 4156   
37             38                   Αγίας Φυλάξεως 230, Κάψαλος 3083   
38             39                          Σαριπόλου 1, Λεμεσός 3036   
39             40         Λεωφ. Φραγκλίνου Ρούσβελτ 86, Λεμεσός 3011   
40             41          Λεωφ. Αρχ. Μακαρίου 121-123, Λεμεσός 3021   
41             42                  Μακεδονίας 54, Μέσα Γειτονιά 4001   
42             43                     Νίκου Παττίχη 39, Λεμεσός 3071   
43             44                        Οδού Πάφου 51, Λεμεσός 3052   
44             45      Αγ.Ζώνης & Θεσσαλονίκης, Πεντάδρομος, Λεμεσός   
45             46                            Ολύμπου 6, Πλάτρες 4820   
46             47                     Γλαύκου Κληρίδη 43, Αγρός 4860   
47             48                        Ακαδημίας 86B, Λεμεσός 4043   
48             49                   Γεωργίου Α' 81, Γερμασόγεια 4047   
49             50  Θεόδωρου Ποταμιάνου 48, Συνοικισμός Μακαρίου Ι...   
50             51                    Ηλία Καννάουρου 33, Ύψωνας 4180   
51             52                    οδός Ακαμαντίδος 26, Πάφος 8016   
52             53                Ελευθερίου Βενιζέλου 49, Πάφος 8021   
53             54             E.Παλληκαρίδη&Α.Τσιέλεπου , Πάφος 8010   
54             55  Λεωφόρος Κόλπου Κοραλίων 

In [61]:
join_df2.info()
join_df2.to_csv('CompA_InitialStatsV3.csv', encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Service Point    60 non-null     int64  
 1   Full Address     60 non-null     object 
 2   Latitude         60 non-null     float64
 3   Longitude        60 non-null     float64
 4   District         60 non-null     object 
 5   NoOfCustomers    60 non-null     int64  
 6   Min Distance     60 non-null     float64
 7   Max Distance     60 non-null     float64
 8   Mean Distance    60 non-null     float64
 9   Median Distance  60 non-null     float64
 10  95th Percentile  60 non-null     float64
 11  5th Percentile   60 non-null     float64
dtypes: float64(8), int64(2), object(2)
memory usage: 6.1+ KB


In [62]:
#Repeat the process to get the metrics for Company B Existing Service Points
#dist_df = df[(df['lat'] <= lat1) & (df['lat'] >= lat2) & (df['lng'] <= lon1) & (df['lng'] >= lon2)]
dfB = pd.read_csv('https://storage.googleapis.com/geolocation_data_project_cm/distances_CompB.csv', delimiter=',', encoding='utf-8')
distB_df = dfB[(dfB['lat'] <= lat1) & (dfB['lat'] >= lat2) & (dfB['lng'] <= lon1) & (dfB['lng'] >= lon2)]
print(distB_df.head(20))
print(distB_df.info())

                                    ORGANISATION_NAME  REGISTRATION_NO  \
0                    THE SUN INSURANCE OFFICE LIMITED               11   
1                     COMPTOIR D' ESCOMPTE DE NICOSIE               13   
2                                    SIMMONS BROTHERS               32   
3                                  NEW STAR SHOE SHOP              118   
4                           CHARALAMBOS PIERI AND SON               17   
5   ΚΥΠΡΙΑΚΟΝ ΕΜΠΟΡΙΚΟΝ ΚΑΙ ΒΙΟΜΗΧΑΝΙΚΟΝ ΕΠΙΜΕΛΗΤΗ...               19   
6   ΕΜΠΟΡΙΚΟΝ ΚΑΙ ΒΙΟΜΗΧΑΝΙΚΟΝ ΕΠΙΜΕΛΗΤΗΡΙΟΝ ΛΕΥΚΩ...              502   
7   Ο ΚΥΠΡΙΑΚΟΣ ΣΥΝΔΕΣΜΟΣ ΙΔΙΟΚΤΗΤΩΝ ΑΚΙΝΗΤΩΝ ΛΙΜΙΤΕΔ             1293   
8                CYPRUS-AMERICAN BUSINESS ASSOCIATION            17284   
9                                               CYABA            17285   
10                                      AMCHAM CYPRUS            35841   
11                                   C.J. CACOYIANNIS               28   
12                                    

In [63]:
msrB_df = distB_df.select_dtypes(include=['number'])
msrB_df = msrB_df.drop(['lat_txt','lng_txt','lat','lng','address_seq_no','REGISTRATION_NO'], axis=1)
msrB_df['min_dist'] = msrB_df.min(axis='columns')
msrB_df['min_sp'] = msrB_df.idxmin(axis='columns')
msrB_df

Valid Seq      1      2      3      4      5      6      7      8  \
0          281510  17.54   2.17   5.36   2.26   7.27   3.50  14.64   0.68   
1          274852  20.12   4.11   8.79   5.01  10.71   5.94  17.02   3.94   
2          274852  20.12   4.11   8.79   5.01  10.71   5.94  17.02   3.94   
3          274852  20.12   4.11   8.79   5.01  10.71   5.94  17.02   3.94   
4          262535  20.12   4.11   8.79   5.01  10.71   5.94  17.02   3.94   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
218733     267985  50.54  62.31  56.85  60.94  54.98  60.84  54.88  61.77   
218734     565102  22.75  37.92  34.37  33.68  35.19  38.52  46.17  36.20   
218735     439357  20.62  13.32  10.98  14.75   9.12  11.49   4.14  13.80   
218736     352550  21.75  39.87  35.21  35.71  35.45  40.04  45.63  38.20   
218737     195348  48.49  61.19  55.59  59.58  53.79  59.78  54.46  60.55   

            9  ...     35     36      37      38      39      40      41  \
0        2.14  ...  60.48  59.29   86.22   94.08   96.56   93.99   95.37   
1        3.19  ...  63.91  62.71   88.66   97.16   99.60   97.07   98.37   
2        3.19  ...  63.91  62.71   88.66   97.16   99.60   97.07   98.37   
3        3.19  ...  63.91  62.71   88.66   97.16   99.60   97.07   98.37   
4        3.19  ...  63.91  62.71   88.66   97.16   99.60   97.07   98.37   
...       ...  ...    ...    ...     ...     ...     ...     ...     ...   
218733  63.04  ...   4.17   5.51   65.24   50.91   54.43   50.91   54.89   
218734  34.17  ...  58.69  57.19  109.61  108.15  111.31  108.10  110.90   
218735  16.35  ...  50.50  49.51   72.09   79.76   82.19   79.67   80.96   
218736  36.73  ...  49.68  48.18  104.15  100.62  103.88  100.58  103.61   
218737  61.67  ...   0.51   1.99   68.70   54.73   58.25   54.73   58.71   

            42  min_dist  min_sp  
0        94.27      0.51      10  
1        97.29      0.51      13  
2        97.29      0.51      13  
3        97.29      0.51      13  
4        97.29      0.51      13  
...        ...       ...     ...  
218733   53.11      1.82      30  
218734  109.39      0.98      25  
218735   79.88      4.14       7  
218736  102.04      2.53      20  
218737   56.93      0.00      27  

[217788 rows x 45 columns]

In [64]:
#Keep only the the interesting columns
serv_ptsB_df = pd.DataFrame(msrB_df, columns=['min_dist', 'min_sp'])
serv_ptsB_df['min_sp'] = serv_ptsB_df['min_sp'].astype('int64')
serv_ptsB_df

min_dist  min_sp
0           0.51      10
1           0.51      13
2           0.51      13
3           0.51      13
4           0.51      13
...          ...     ...
218733      1.82      30
218734      0.98      25
218735      4.14       7
218736      2.53      20
218737      0.00      27

[217788 rows x 2 columns]

In [65]:
#Concatenate dataframes to get the closest service point per customer
custB_df = pd.DataFrame(distB_df, columns=['ORGANISATION_NAME','address_seq_no', 'FULL ADDRESS', 'lat', 'lng'])
custB_sp_df = pd.concat([custB_df,serv_ptsB_df],axis=1)
custB_sp_df.rename(columns={'address_seq_no':'AddressID'}, inplace=True)
custB_sp_df.rename(columns={'min_dist':'MinDistTo_CompB_SP'}, inplace=True)
custB_sp_df.rename(columns={'min_sp':'CompB_SP'}, inplace=True)
custB_sp_df

ORGANISATION_NAME  AddressID  \
0       THE SUN INSURANCE OFFICE LIMITED     281510   
1        COMPTOIR D' ESCOMPTE DE NICOSIE     274852   
2                       SIMMONS BROTHERS     274852   
3                     NEW STAR SHOE SHOP     274852   
4              CHARALAMBOS PIERI AND SON     262535   
...                                  ...        ...   
218733   BEAUTY STUDIO BY THEKLA LIMITED     267985   
218734                      ZI. RA 1 LTD     565102   
218735                 ACLA AUDITORS LTD     439357   
218736         ΣΥΛΒΕΣΤΡΟΥ ΚΑΙ ΠΗΛΑΔΗ ΛΤΔ     352550   
218737                       ILLIDAN LTD     195348   

                                             FULL ADDRESS        lat  \
0                    ΠΡΙΓΚΙΠΙΣΣΗΣ ΝΤΕ ΤΥΡΑΣ 12  ΛΕΥΚΩΣΙΑ   35.160221   
1                                               ΛΕΥΚΩΣΙΑ   35.185566   
2                                               ΛΕΥΚΩΣΙΑ   35.185566   
3                                               ΛΕΥΚΩΣΙΑ   35.185566   
4                               ARISTIDES ROAD  ΛΕΥΚΩΣΙΑ   35.185566   
...                                                   ...        ...   
218733  ΠΑΝΑΓΙΩΤΗ ΑΝΑΓΝΩΣΤΟΠΟΥΛΟΥ 68  ΚΑΤΩ ΠΟΛΕΜΙΔΙΑ 4...  34.685159   
218734                        ΓΛΑΔΣΤΩΝΟΣ 1  6023 ΛΑΡΝΑΚΑ   34.918222   
218735   ΠΕΝΤΑΔΑΚΤΥΛΟΥ 21  ΑΓΙΟΙ ΤΡΙΜΙΘΙΑΣ 2671 ΛΕΥΚΩΣΙΑ   35.114799   
218736              ΟΔΥΣΣΕΩΣ 10  ΤΕΡΣΕΦΑΝΟΥ 7562 ΛΑΡΝΑΚΑ   34.854814   
218737                     ΑΡΓΥΡΗ ΓΑΒΑΛΑ 3  3112 ΛΕΜΕΣΟΣ   34.678632   

              lng  MinDistTo_CompB_SP  CompB_SP  
0       33.360473                0.51        10  
1       33.382276                0.51        13  
2       33.382276                0.51        13  
3       33.382276                0.51        13  
4       33.382276                0.51        13  
...           ...                 ...       ...  
218733  33.000279                1.82        30  
218734  33.620063                0.98        25  
218735  33.211395                4.14         7  
218736  33.546889                2.53        20  
218737  33.041305                0.00        27  

[217788 rows x 7 columns]

In [66]:
custB_sp_df.to_csv('CompB_MinSP_AllData.csv', encoding='utf-8')

In [67]:
CompB_Loc = pd.read_csv('https://storage.googleapis.com/geolocation_data_project_cm/CompB_Locs.csv', delimiter=';', encoding='utf-8')
CompB_Loc.dropna(inplace=True)
CompB_Loc['AA'] = CompB_Loc['AA'].astype('int64')
CompB_Loc.rename(columns={'AA': 'Service Point'}, inplace=True)
CompB_Loc

Service Point                                       Full Address  \
0               1     1, Synergatismou Str., Pera Chorio Nisou, 2572   
1               2  10-12, Grigori Afxentiou Ave., Agios Dometios,...   
2               3               124, Tseriou Avenue, Strovolos, 2044   
3               4               173, Athalassa Ave., Strovolos, 2025   
4               5      200, Arch. Makariou III Ave., Lakatamia, 2311   
5               6                        3, Aheon Str., Engomi, 2413   
6               7  31, Georgiou Griva Digeni Str., Kokkinotrimith...   
7               8        34, Demostheni Severi Avenue, Nicosia, 1080   
8               9                 43 Aglantzias Ave. Aglantzia, 2108   
9              10               5, Esperidon Street, Strovolos, 2001   
10             11              56, Arch. Makariou Ave. 1075, Nicosia   
11             12               60, Griva Digheni Str., 2614 Arediou   
12             13           66, Agiou Ilarionos Ave., Kaimakli, 1026   
13             14                  71, Stavros Ave., Strovolos, 2035   
14             15        85, Giannou Kranidioti Avenue, Latsia, 2231   
15             16              92, Digeni Akrita Ave., Nicosia, 1061   
16             17           54, Tefkrou Anthia Str., Ayia Napa, 5330   
17             18  Corner Protaras & 2-4 Megalou Alexandrou Ave.,...   
18             19          11, Michail Karaoli Str., Aradippou, 7104   
19             20                  2, Dromolaxias Avenue, Kiti, 7550   
20             21                    2, Kalymnou Str., Larnaca, 6037   
21             22             2, Zenonos Kitieus Str., Larnaca, 6023   
22             23             53, Spyrou Kyprianou ave. Larnaca 6051   
23             24  66, Arch. Makariou III Avenue, Livadia Larnaka...   
24             25      68, Georgiou Griva Digeni Ave., Larnaca, 6046   
25             26  1, Corner Spyrou Kyprianou & Synergatismou, Me...   
26             27      108, Fr.Roosevelt Str.&Avanas, Limassol, 3011   
27             28            13, Stavrodromiou Str., Kalavasos, 7733   
28             29  131, Arch.Makariou III Ave. & Ioanni Polemi, L...   
29             30                    15, Paphos Str., Limassol, 3052   
30             31      232 Z, Arch. Leontiou A' Str., Limassol, 3020   
31             32            24, Ilia Kannaourou Str., Ypsonas, 4187   
32             33           296, Agias Fylaxeos Str., Limassol, 3116   
33             34      42, Spyrou Kyprianou Avenue, Yermasogia, 4043   
34             35                97, Gladstonos Str., Limassol, 3032   
35             36  Arch. Makariou 229 & John Kennedy, Limassol, 3106   
36             37      1, Kyproleontos Str., Polis Chrysochous, 8820   
37             38       15, Georgiou Griva Digeni Ave., Paphos, 8047   
38             39          26, Tombs of the Kings Ave., Paphos, 8046   
39             40  37, Archiepiskopou Makariou III Ave., 8200, Ge...   
40             41            76, Eleftherias Avenue, Chlorakas, 8220   
41             42                       Ellados Ave. 79, 8020 Paphos   

     Latitude  Longitude     District  
0   35.004875  33.393852      Nicosia  
1   35.171138  33.340676      Nicosia  
2   35.118028  33.332058      Nicosia  
3   35.141553  33.370235      Nicosia  
4   35.108642  33.311404      Nicosia  
5   35.164350  33.322300      Nicosia  
6   35.150780  33.199900      Nicosia  
7   35.159420  33.353050      Nicosia  
8   35.156930  33.383700      Nicosia  
9   35.156550  33.363760      Nicosia  
10  35.161000  33.368980      Nicosia  
11  35.056240  33.205590      Nicosia  
12  35.186410  33.387770      Nicosia  
13  35.134820  33.358080      Nicosia  
14  35.100100  33.379700      Nicosia  
15  35.167700  33.371050      Nicosia  
16  34.993100  33.986030  Ammochostos  
17  35.044080  33.998170  Ammochostos  
18  34.948752  33.593616      Larnaca  
19  34.846478  33.572730      Larnaca  
20  34.916606  33.606429      Larnaca  
21  34.917684  33.635243   

In [68]:
#Some baseline counts regarding existing physical branch network of Company B
customers_per_spB = serv_ptsB_df.groupby("min_sp").count().reset_index()
customers_per_spB.columns = ['Service Point', 'NoOfCustomers']
customers_per_spB['Service Point'] = customers_per_spB['Service Point'].astype('int64')

join_dfB = pd.merge(customers_per_spB,CompB_Loc,on="Service Point",how="inner")
join_dfB.sort_values('NoOfCustomers', ascending=False)
join_dfB

Service Point  NoOfCustomers  \
0               1           2885   
1               2           4242   
2               3           3207   
3               4           2912   
4               5           3047   
5               6           6953   
6               7           1384   
7               8          22259   
8               9           3739   
9              10           9068   
10             11           6266   
11             12           1074   
12             13           5008   
13             14           6025   
14             15           4321   
15             16          19724   
16             17           2860   
17             18           6412   
18             19           2061   
19             20           2833   
20             21           2829   
21             22           9098   
22             23           2397   
23             24           2038   
24             25           4841   
25             26           5193   
26             27          12147   
27             28            743   
28             29           5862   
29             30           6546   
30             31           2494   
31             32           1073   
32             33           7840   
33             34           5405   
34             35          10804   
35             36           8217   
36             37            985   
37             38            818   
38             39           5708   
39             40            581   
40             41           1992   
41             42           3897   

                                         Full Address   Latitude  Longitude  \
0      1, Synergatismou Str., Pera Chorio Nisou, 2572  35.004875  33.393852   
1   10-12, Grigori Afxentiou Ave., Agios Dometios,...  35.171138  33.340676   
2                124, Tseriou Avenue, Strovolos, 2044  35.118028  33.332058   
3                173, Athalassa Ave., Strovolos, 2025  35.141553  33.370235   
4       200, Arch. Makariou III Ave., Lakatamia, 2311  35.108642  33.311404   
5                         3, Aheon Str., Engomi, 2413  35.164350  33.322300   
6   31, Georgiou Griva Digeni Str., Kokkinotrimith...  35.150780  33.199900   
7         34, Demostheni Severi Avenue, Nicosia, 1080  35.159420  33.353050   
8                  43 Aglantzias Ave. Aglantzia, 2108  35.156930  33.383700   
9                5, Esperidon Street, Strovolos, 2001  35.156550  33.363760   
10              56, Arch. Makariou Ave. 1075, Nicosia  35.161000  33.368980   
11               60, Griva Digheni Str., 2614 Arediou  35.056240  33.205590   
12           66, Agiou Ilarionos Ave., Kaimakli, 1026  35.186410  33.387770   
13                  71, Stavros Ave., Strovolos, 2035  35.134820  33.358080   
14        85, Giannou Kranidioti Avenue, Latsia, 2231  35.100100  33.379700   
15              92, Digeni Akrita Ave., Nicosia, 1061  35.167700  33.371050   
16           54, Tefkrou Anthia Str., Ayia Napa, 5330  34.993100  33.986030   
17  Corner Protaras & 2-4 Megalou Alexandrou Ave.,...  35.044080  33.998170   
18          11, Michail Karaoli Str., Aradippou, 7104  34.948752  33.593616   
19                  2, Dromolaxias Avenue, Kiti, 7550  34.846478  33.572730   
20                    2, Kalymnou Str., Larnaca, 6037  34.916606  33.606429   
21             2, Zenonos Kitieus Str., Larnaca, 6023  34.917684  33.635243   
22             53, Spyrou Kyprianou ave. Larnaca 6051  34.930230  33.622730   
23  66, Arch. Makariou III Avenue, Livadia Larnaka...  34.952730  33.625890   
24      68, Georgiou Griva Digeni Ave., Larnaca, 6046  34.923320  33.611250   
25  1, Corner Spyrou Kyprianou & Synergatismou, Me...  34.698427  33.049261   
26      108, Fr.Roosevelt Str.&Avanas, Limassol, 3011  34.678632  33.041305   
27            13, Stavrodromiou Str., Kalavasos, 7733  34.772429  33.295977   
28  131, Arch.Makariou III Ave. & Ioanni Polemi, L...  34.688449  33.032908   
29                    15, Paphos Str., Limassol, 3052  34.671807  33.011860   
30    

In [69]:
#Statistics per service point
stats_per_spB = serv_ptsB_df.groupby(['min_sp']).aggregate([np.min,np.max,np.mean,
                                                            np.median]).reset_index()

stats_per_spB.columns = stats_per_spB.columns.map('_'.join)
stats_per_spB.rename(columns={'min_sp_':'Service Point'}, inplace=True)
stats_per_spB.rename(columns={'min_dist_amin':'Min Distance'}, inplace=True)
stats_per_spB.rename(columns={'min_dist_amax':'Max Distance'}, inplace=True)
stats_per_spB.rename(columns={'min_dist_mean':'Mean Distance'}, inplace=True)
stats_per_spB.rename(columns={'min_dist_median':'Median Distance'}, inplace=True)

stats_per_spB.sort_values('Mean Distance')
stats_per_spB

Service Point  Min Distance  Max Distance  Mean Distance  Median Distance
0               1          0.00         13.34       4.298163            2.830
1               2          0.00          3.03       0.679144            0.620
2               3          0.00          3.29       1.371204            1.300
3               4          0.00          1.60       0.561868            0.530
4               5          0.00          5.01       1.896242            1.640
5               6          0.03          4.30       1.222766            1.170
6               7          0.14         33.08       6.215708            4.140
7               8          0.01          2.33       0.801802            0.850
8               9          0.00         20.78       1.119660            0.890
9              10          0.03          1.35       0.631859            0.560
10             11          0.02          0.96       0.515932            0.560
11             12          0.40         35.10      14.073799           10.240
12             13          0.05         14.40       0.677474            0.510
13             14          0.01          2.48       1.306013            1.420
14             15          0.19         11.38       1.962708            2.230
15             16          0.01          2.46       0.863439            0.980
16             17          0.10         14.16       4.648678            1.390
17             18          0.00         25.23       3.673539            2.020
18             19          0.12         14.07       4.801266            1.300
19             20          0.00         12.05       5.140805            7.230
20             21          0.00          5.08       0.542881            0.270
21             22          0.00          3.22       0.787174            0.820
22             23          0.10          1.69       0.704618            0.630
23             24          0.10         23.96       8.058842            6.890
24             25          0.11          1.72       0.899118            0.980
25             26          0.00          3.16       0.626176            0.500
26             27          0.00          1.45       0.281485            0.070
27             28          0.00         18.27       7.660713            8.060
28             29          0.00          2.06       0.663794            0.690
29             30          0.00          2.68       1.024704            0.910
30             31          0.00         35.24      10.467394            3.490
31             32          0.00          3.34       1.753756            1.810
32             33          0.06         24.98       5.108962            6.720
33             34          0.04         18.26       1.967748            1.120
34             35          0.01          0.97       0.442479            0.490
35             36          0.00          1.35       0.531127            0.620
36             37          0.00         34.79       5.110650            0.280
37             38          0.00         21.97       3.690024            1.255
38             39          0.00          2.32       0.994085            0.940
39             40          0.05         24.46       2.867728            0.860
40             41          0.05         13.35       3.897254            2.000
41             42          0.02         21.90       1.642599            1.130

In [70]:
stats_per_spB['95th Percentile'] = custB_sp_df.groupby('CompB_SP')['MinDistTo_CompB_SP'].quantile(0.95)
stats_per_spB['5th Percentile'] = custB_sp_df.groupby('CompB_SP')['MinDistTo_CompB_SP'].quantile(0.05)
stats_per_spB

Service Point  Min Distance  Max Distance  Mean Distance  Median Distance  \
0               1          0.00         13.34       4.298163            2.830   
1               2          0.00          3.03       0.679144            0.620   
2               3          0.00          3.29       1.371204            1.300   
3               4          0.00          1.60       0.561868            0.530   
4               5          0.00          5.01       1.896242            1.640   
5               6          0.03          4.30       1.222766            1.170   
6               7          0.14         33.08       6.215708            4.140   
7               8          0.01          2.33       0.801802            0.850   
8               9          0.00         20.78       1.119660            0.890   
9              10          0.03          1.35       0.631859            0.560   
10             11          0.02          0.96       0.515932            0.560   
11             12          0.40         35.10      14.073799           10.240   
12             13          0.05         14.40       0.677474            0.510   
13             14          0.01          2.48       1.306013            1.420   
14             15          0.19         11.38       1.962708            2.230   
15             16          0.01          2.46       0.863439            0.980   
16             17          0.10         14.16       4.648678            1.390   
17             18          0.00         25.23       3.673539            2.020   
18             19          0.12         14.07       4.801266            1.300   
19             20          0.00         12.05       5.140805            7.230   
20             21          0.00          5.08       0.542881            0.270   
21             22          0.00          3.22       0.787174            0.820   
22             23          0.10          1.69       0.704618            0.630   
23             24          0.10         23.96       8.058842            6.890   
24             25          0.11          1.72       0.899118            0.980   
25             26          0.00          3.16       0.626176            0.500   
26             27          0.00          1.45       0.281485            0.070   
27             28          0.00         18.27       7.660713            8.060   
28             29          0.00          2.06       0.663794            0.690   
29             30          0.00          2.68       1.024704            0.910   
30             31          0.00         35.24      10.467394            3.490   
31             32          0.00          3.34       1.753756            1.810   
32             33          0.06         24.98       5.108962            6.720   
33             34          0.04         18.26       1.967748            1.120   
34             35          0.01          0.97       0.442479            0.490   
35             36          0.00          1.35       0.531127            0.620   
36             37          0.00         34.79       5.110650            0.280   
37             38          0.00         21.97       3.690024            1.255   
38             39          0.00          2.32       0.994085            0.940   
39             40          0.05         24.46       2.867728            0.860   
40             41          0.05         13.35       3.897254            2.000   
41             42          0.02         21.90       1.642599            1.130   

    95th Percentile  5th Percentile  
0               NaN             NaN  
1             10.27            0.02  
2              1.07            0.24  
3              2.63            0.45  
4              0.90            0.12  
5              4.38            0.00  
6              2.64            0.29  
7             15.10            0.22  
8              1.25            0.22  
9              1.94            0.44  
10             1.23            0.31  
11             0.87            0.16  
12            29.17       

In [71]:
#Join statistics with the main service point information
join_df2B = pd.merge(join_dfB,stats_per_spB,on="Service Point",how="inner")
join_df2B.sort_values('NoOfCustomers', ascending=False)

Service Point  NoOfCustomers  \
7               8          22259   
15             16          19724   
26             27          12147   
34             35          10804   
21             22           9098   
9              10           9068   
35             36           8217   
32             33           7840   
5               6           6953   
29             30           6546   
17             18           6412   
10             11           6266   
13             14           6025   
28             29           5862   
38             39           5708   
33             34           5405   
25             26           5193   
12             13           5008   
24             25           4841   
14             15           4321   
1               2           4242   
41             42           3897   
8               9           3739   
2               3           3207   
4               5           3047   
3               4           2912   
0               1           2885   
16             17           2860   
19             20           2833   
20             21           2829   
30             31           2494   
22             23           2397   
18             19           2061   
23             24           2038   
40             41           1992   
6               7           1384   
11             12           1074   
31             32           1073   
36             37            985   
37             38            818   
27             28            743   
39             40            581   

                                         Full Address   Latitude  Longitude  \
7         34, Demostheni Severi Avenue, Nicosia, 1080  35.159420  33.353050   
15              92, Digeni Akrita Ave., Nicosia, 1061  35.167700  33.371050   
26      108, Fr.Roosevelt Str.&Avanas, Limassol, 3011  34.678632  33.041305   
34                97, Gladstonos Str., Limassol, 3032  34.681440  33.045650   
21             2, Zenonos Kitieus Str., Larnaca, 6023  34.917684  33.635243   
9                5, Esperidon Street, Strovolos, 2001  35.156550  33.363760   
35  Arch. Makariou 229 & John Kennedy, Limassol, 3106  34.687060  33.060540   
32           296, Agias Fylaxeos Str., Limassol, 3116  34.705090  33.026760   
5                         3, Aheon Str., Engomi, 2413  35.164350  33.322300   
29                    15, Paphos Str., Limassol, 3052  34.671807  33.011860   
17  Corner Protaras & 2-4 Megalou Alexandrou Ave.,...  35.044080  33.998170   
10              56, Arch. Makariou Ave. 1075, Nicosia  35.161000  33.368980   
13                  71, Stavros Ave., Strovolos, 2035  35.134820  33.358080   
28  131, Arch.Makariou III Ave. & Ioanni Polemi, L...  34.688449  33.032908   
38          26, Tombs of the Kings Ave., Paphos, 8046  34.770075  32.413756   
33      42, Spyrou Kyprianou Avenue, Yermasogia, 4043  34.703800  33.081530   
25  1, Corner Spyrou Kyprianou & Synergatismou, Me...  34.698427  33.049261   
12           66, Agiou Ilarionos Ave., Kaimakli, 1026  35.186410  33.387770   
24      68, Georgiou Griva Digeni Ave., Larnaca, 6046  34.923320  33.611250   
14        85, Giannou Kranidioti Avenue, Latsia, 2231  35.100100  33.379700   
1   10-12, Grigori Afxentiou Ave., Agios Dometios,...  35.171138  33.340676   
41                       Ellados Ave. 79, 8020 Paphos  34.785930  32.432190   
8                  43 Aglantzias Ave. Aglantzia, 2108  35.156930  33.383700   
2                124, Tseriou Avenue, Strovolos, 2044  35.118028  33.332058   
4       200, Arch. Makariou III Ave., Lakatamia, 2311  35.108642  33.311404   
3                173, Athalassa Ave., Strovolos, 2025  35.141553  33.370235   
0      1, Synergatismou Str., Pera Chorio Nisou, 2572  35.004875  33.393852   
16           54, Tefkrou Anthia Str., Ayia Napa, 5330  34.993100  33.986030   
19                  2, Dromolaxias Avenue, Kiti, 7550  34.846478  33.572730   
20                    2, Kalymnou Str., Larnaca, 6037  34.916606  33.606429   
30    

In [73]:
join_df2B.info()
join_df2B.to_csv('CompB_InitialStats.csv', encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Service Point    42 non-null     int64  
 1   NoOfCustomers    42 non-null     int64  
 2   Full Address     42 non-null     object 
 3   Latitude         42 non-null     float64
 4   Longitude        42 non-null     float64
 5   District         42 non-null     object 
 6   Min Distance     42 non-null     float64
 7   Max Distance     42 non-null     float64
 8   Mean Distance    42 non-null     float64
 9   Median Distance  42 non-null     float64
 10  95th Percentile  41 non-null     float64
 11  5th Percentile   41 non-null     float64
dtypes: float64(8), int64(2), object(2)
memory usage: 4.3+ KB
